In [3]:
import pandas as pd
import numpy
import os
from matplotlib import pyplot as plt
import seaborn as sns
import glob

In [2]:
datPath = os.getcwd() + '/Data/'


/Users/darrensunart/Projects/elearning-covid19/Data/


# Take a look at engagement data
### Engagement data
The engagement data are aggregated at school district level, and each file in the folder `engagement_data` represents data from one school district. The 4-digit file name represents `district_id` which can be used to link to district information in `district_info.csv`. The `lp_id` can be used to link to product information in `product_info.csv`.

| Name | Description |
| :--- | :----------- |
| time | date in "YYYY-MM-DD" |
| lp_id | The unique identifier of the product |
| pct_access | Percentage of students in the district have at least one page-load event of a given product and on a given day |
| engagement_index | Total page-load events per one thousand students of a given product and on a given day |


In [43]:
engagement_files = glob.glob(datPath + '/engagement_data/*.csv')

dfs = []

for file in engagement_files:
    
    df = pd.read_csv(file)
    district_id = file.split('/')[-1].split('.')[0]
    df['district_id'] = [district_id]*len(df)
    dfs.append(df)
engagement_df = pd.concat(dfs,axis=0).dropna()
engagement_df.lp_id = engagement_df.lp_id.astype(int)
engagement_df.district_id = engagement_df.district_id.astype(int)
engagement_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16945301 entries, 0 to 36572
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   time              object 
 1   lp_id             int64  
 2   pct_access        float64
 3   engagement_index  float64
 4   district_id       int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 775.7+ MB


In [44]:
product_info = pd.read_csv(datPath + '/products_info.csv')
product_info = product_info.rename(columns={'LP ID':'lp_id'})
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   lp_id                       372 non-null    int64 
 1   URL                         372 non-null    object
 2   Product Name                372 non-null    object
 3   Provider/Company Name       371 non-null    object
 4   Sector(s)                   352 non-null    object
 5   Primary Essential Function  352 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.6+ KB


In [45]:
engagement_product = engagement_df.merge(product_info, on='lp_id', how='inner')
engagement_product.head(10)

,time,lp_id,pct_access,engagement_index,district_id,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,2020-01-01,29322,0.04,1.30,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
1,2020-01-02,29322,0.26,17.78,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
2,2020-01-03,29322,0.82,121.42,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
3,2020-01-06,29322,2.86,451.43,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,2020-01-07,29322,2.64,269.50,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
5,2020-01-08,29322,3.72,465.57,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
6,2020-01-09,29322,1.99,171.00,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
7,2020-01-10,29322,6.31,1011.25,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
8,2020-01-12,29322,0.04,1.73,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms
9,2020-01-13,29322,1.51,139.71,3188,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms


In [46]:
district_info = pd.read_csv(datPath + '/districts_info.csv')
district_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   district_id               233 non-null    int64 
 1   state                     176 non-null    object
 2   locale                    176 non-null    object
 3   pct_black/hispanic        176 non-null    object
 4   pct_free/reduced          148 non-null    object
 5   county_connections_ratio  162 non-null    object
 6   pp_total_raw              118 non-null    object
dtypes: int64(1), object(6)
memory usage: 12.9+ KB


In [48]:
district_info.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
1,2685,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
3,3188,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
engagement_product_district_df = engagement_product.merge(district_info, on = 'district_id', how = 'inner')
engagement_product_district_df.loc[engagement_product_district_df.district_id!=3188].head(50)

,time,lp_id,pct_access,engagement_index,district_id,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
36185,2020-01-02,29322,0.34,6.85,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36186,2020-01-03,29322,0.07,4.11,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36187,2020-01-05,29322,0.07,0.68,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36188,2020-01-06,29322,0.21,10.27,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36189,2020-01-07,29322,0.34,7.53,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36190,2020-01-08,29322,0.68,51.37,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36191,2020-01-10,29322,0.21,2.05,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36192,2020-01-13,29322,0.07,0.68,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36193,2020-01-14,29322,0.14,1.37,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
36194,2020-01-15,29322,1.16,137.67,7305,http://www.khanacademy.org,Khan Academy,Khan Academy,PreK-12; Higher Ed,LC - Digital Learning Platforms,Massachusetts,Suburb,"[0, 0.2[",NaN,"[0.18, 1[","[14000, 16000["
